In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
import distutils

print('현재 경기 상황을 입력하세요.\n')
print('입력할 수 있는 경기 상황은 다음과 같습니다.\n')
print(' firstBlood_ours\n', 'firstTowerKill_ours\n', 'firstInhibitorKill_ours\n', 'firstBaron_ours\n', 'firstDragon_ours\n',
     'firstRiftHerald_ours\n', '4dragons_ours\n','olddragon_ours\n','kda_ours\n', 'kda_enemy\n','level_diff_ours_minus_enemy\n',
     'inhibitorKills_gap_ours_minus_enemy\n', 'baronKills_gap_ours_minus_enemy\n', 'dragonKills_gap_ours_minus_enemy\n',
     'riftHeraldKills_gap_ours_minus_enemy\n')

print("경기 상황 입력을 마치면 '입력완료'라고 입력해주세요.\n")

option_dict = {}
total_option_ls = ['firstBlood_ours', 'firstTowerKill_ours', 'firstInhibitorKill_ours', 'firstBaron_ours', 'firstDragon_ours',
     'firstRiftHerald_ours', '4dragons_ours','olddragon_ours','kda_ours', 'kda_enemy','level_diff_ours_minus_enemy',
     'inhibitorKills_gap_ours_minus_enemy', 'baronKills_gap_ours_minus_enemy', 'dragonKills_gap_ours_minus_enemy',
     'riftHeraldKills_gap_ours_minus_enemy']

while True:
    option = input('조건을 주고자 하는 경기 상황\n')
    
    if option == '입력완료':
        break
        
    if option in ['firstBlood_ours', 'firstTowerKill_ours', 'firstInhibitorKill_ours',
                  'firstBaron_ours', 'firstDragon_ours','firstRiftHerald_ours', '4dragons_ours', 'olddragon_ours'] :
        
        state = input('값을 입력해주세요.(ex) True/False)\n')
        
        if state == '입력완료':
            break

        value = [bool(distutils.util.strtobool(state))]


    
    if option in ['kda_ours', 'kda_enemy', 'level_diff_ours_minus_enemy', 'inhibitorKills_gap_ours_minus_enemy', 'baronKills_gap_ours_minus_enemy',
                 'dragonKills_gap_ours_minus_enemy', 'riftHeraldKills_gap_ours_minus_enemy']:
        
        state = input('값을 입력해주세요. (ex)3)\n')
        value = [float(state)]
                      
        if state == '입력완료':
            break
            
            
    elif option not in total_option_ls:
        print('옵션을 올바르게 입력해주세요!\n')
        
        
    option_dict[option] = value
    
option_df = pd.DataFrame.from_dict(option_dict)


model_data = pd.read_csv('model_data.csv')
model_data = model_data[['win_blue'] + option_df.columns.tolist()]
train, test = train_test_split(model_data, test_size=0.25, random_state=1234)

X_train, y_train = train.drop('win_blue', axis=1), train['win_blue']
X_test, y_test = test.drop('win_blue', axis=1), test['win_blue']


rf = RandomForestClassifier(n_estimators=100, random_state=123456)
rf.fit(X_train, y_train)

predicted1 = rf.predict(X_test)
accuracy1 = accuracy_score(y_test, predicted1)


print('\n\n총 ',X_train.shape[0],'개의 경기를 학습하고 ', X_test.shape[0], '개의 경기를 예측해 모델의 성능을 측정합니다.')

print('\n랜덤포레스트를 사용했을 때, 경기 승리 확률은 ',round(rf.predict_proba(option_df)[:,1][0]*100,2),'% 입니다.' )
print(f'사용된 랜덤포레스트 모델의 성능은 정확도 {accuracy1*100:.4} % 입니다.')


all_variables = '+'.join(X_train.columns)
formula = 'win_blue~' + all_variables
glm = smf.glm(formula=formula, data=train, family=sm.families.Binomial()) .fit()
glm.summary()

predicted2 = glm.predict(test)
accuracy2 = accuracy_score(y_test,np.where(predicted2 >0.5, 1,0))
ci_lower = glm.get_prediction(option_df).summary_frame()[['mean_ci_lower', 'mean_ci_upper']]['mean_ci_lower'][0]
ci_upper = glm.get_prediction(option_df).summary_frame()[['mean_ci_lower', 'mean_ci_upper']]['mean_ci_upper'][0]

print('\nGeneralized Linear Model을 사용했을 때, 경기 승리 확률은 ',round(glm.predict(option_df)[0]*100,2),'% 입니다.' )
print('승리 확률의 95% 신뢰구간은 (',round(ci_lower * 100,2),'%, ',round(ci_upper*100,2),'%)',' 입니다')
print(f'사용된 GLM모델의 성능은 정확도 {accuracy2*100:.4} % 입니다.')

현재 경기 상황을 입력하세요.

입력할 수 있는 경기 상황은 다음과 같습니다.

 firstBlood_ours
 firstTowerKill_ours
 firstInhibitorKill_ours
 firstBaron_ours
 firstDragon_ours
 firstRiftHerald_ours
 4dragons_ours
 olddragon_ours
 kda_ours
 kda_enemy
 level_diff_ours_minus_enemy
 inhibitorKills_gap_ours_minus_enemy
 baronKills_gap_ours_minus_enemy
 dragonKills_gap_ours_minus_enemy
 riftHeraldKills_gap_ours_minus_enemy

경기 상황 입력을 마치면 '입력완료'라고 입력해주세요.

